In [32]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from quantfreedom.enums import CandleBodyType
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.indicators.tv_indicators import macd_tv, ema_tv

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
candles = dl_ex_candles(
    exchange='mufex',
    symbol='BTCUSDT',
    timeframe='5m',
    candles_to_dl=1500,
)

In [34]:
datetimes = candles[:, CandleBodyType.Timestamp].astype("datetime64[ms]")
open_prices = candles[:, CandleBodyType.Open]
high_prices = candles[:, CandleBodyType.High]
low_prices = candles[:, CandleBodyType.Low]
closing_prices = candles[:, CandleBodyType.Close]

In [35]:
ema_fast_length = 10
ema_mid_length = 20
ema_slow_length = 200

In [36]:
ema_fast = ema_tv(
    source=closing_prices,
    length=ema_fast_length,
)
ema_mid = ema_tv(
    source=closing_prices,
    length=ema_mid_length,
)
ema_slow = ema_tv(
    source=closing_prices,
    length=ema_slow_length,
)

In [37]:
prev_ema_fast = np.roll(ema_fast, 1)
prev_ema_fast[0] = np.nan

prev_ema_mid = np.roll(ema_mid, 1)
prev_ema_mid[0] = np.nan

In [38]:
fast_above_mid = ema_fast > ema_mid

prev_fast_below_mid = prev_ema_fast < prev_ema_mid

In [39]:
mid_above_slow = ema_mid > ema_slow

low_prices_above_slow = low_prices > ema_slow

entries = (
    (fast_above_mid == True)
    & (prev_fast_below_mid == True)
    & (mid_above_slow == True)
    & (low_prices_above_slow == True)
)

In [40]:
entry_signals = np.where(entries, ema_fast, np.nan)

In [41]:
go.Figure(
    data=[
        go.Candlestick(
            x=datetimes,
            open=open_prices,
            high=high_prices,
            low=low_prices,
            close=closing_prices,
            name="Candles",
        ),
        go.Scatter(
            x=datetimes,
            y=entry_signals,
            mode="markers",
            name="Entries",
            marker=dict(
                size=15,
                symbol="circle",
                color="#00F6FF",
                line=dict(
                    width=1,
                    color="DarkSlateGrey",
                ),
            ),
        ),
    ]
).update_layout(height=800, xaxis_rangeslider_visible=False)

In [42]:
mid_below_slow = ema_mid < ema_slow

high_prices_below_slow = high_prices < ema_slow

entries = (
    (fast_above_mid == True)
    & (prev_fast_below_mid == True)
    & (mid_below_slow == True)
    & (high_prices_below_slow == True)
)
entry_signals = np.where(entries, ema_fast, np.nan)
go.Figure(
    data=[
        go.Candlestick(
            x=datetimes,
            open=open_prices,
            high=high_prices,
            low=low_prices,
            close=closing_prices,
            name="Candles",
        ),
        go.Scatter(
            x=datetimes,
            y=ema_fast,
            name="EMA Fast",
            line_color="lightblue",
        ),
        go.Scatter(
            x=datetimes,
            y=ema_mid,
            name="EMA Mid",
            line_color="yellow",
        ),
        go.Scatter(
            x=datetimes,
            y=ema_slow,
            name="EMA Slow",
            line_color="orange",
        ),
        go.Scatter(
            x=datetimes,
            y=entry_signals,
            mode="markers",
            name="Entries",
            marker=dict(
                size=15,
                symbol="circle",
                color="#00F6FF",
                line=dict(
                    width=1,
                    color="DarkSlateGrey",
                ),
            ),
        ),
    ]
).update_layout(height=800, xaxis_rangeslider_visible=False)